In [1]:
from bs4 import BeautifulSoup as bs
import requests
from textwrap import fill as wrap

In [2]:
url = "https://www.dnd-spells.com/spell/wall-of-fire"
# url = 'https://www.dnd-spells.com/spell/leomunds-secret-chest'

In [3]:
# url = "https://www.roll20.net/Fireball"

In [4]:
from lookup_5e import Roll20Spell

In [47]:
class NewSpell(Roll20Spell):
    def __init__(self, name, site="https://www.dnd-spells.com/spell/"):
        self.name = name.rstrip().title()
        self.attributes = {}
        formatted_name = self.name.lower().replace(' ', '-')
        url = site + formatted_name
        page = requests.get(url, timeout=1)
        if (page.status_code != 200) or (page.url != url):
            raise IOError('{:s} not found at {:s}.'.format(name,
                                                           url))
        soup = bs(page.text, 'html.parser')
        base = soup.find_all('h1')[0]
        school = base.find_next('p')
        self.attributes['School'] = school.text
        body = school.find_next('p')
        details = body.text.strip().replace('\r', '').replace('\n', '').split('                 ')
        for entry in details:
            k, v = entry.split(': ')
            self.attributes[k.title()] = v.capitalize()
        if '(' in self.attributes['Components']:
            self.attributes['Components'], mat = self.attributes['Components'].split('(')
            self.attributes['Material'] = mat.strip(')').capitalize()

        self.attributes['Components'] = self.attributes['Components'].replace(' ', '').replace(',', ' ')
        if 'cantrip' in self.attributes['Level'].lower():
            self.attributes['Level'] = '0'
        desc = body.find_next('p')
        self.desc = desc.text.strip().replace('\r', '')
        if 'higher level' in desc.find_next('h4').text.lower():
            desc = desc.find_next('p')
            self.desc += '\n\nAt Higher Levels: '
            self.desc += desc.text.strip()
        self.desc = wrap(self.desc, 80, drop_whitespace=False,
                        replace_whitespace=False)
        classes = desc.find_next('p').find_next('p')
        self.attributes['Classes'] = ', '.join(classes.text.replace(',', '').split()[1:-1])

In [48]:
name = 'fireball'

In [49]:
a = NewSpell(name)

In [42]:
b = Roll20Spell(name)

In [50]:
print(a.as_dungeonsheets_class())

class Fireball(Spell):
    """A bright streak flashes from your pointing finger to a point you choose within 
    range then blossoms with a low roar into an explosion of flame.
    Each creature in
     a 20-foot radius must make a Dexterity saving throw. A target takes 8d6 fire 
    damage on a failed save, or half as much damage on a successful one. The fire 
    spreads around corners. It ignites flammable objects in the area that aren’t 
    being worn or carried.
    
    At Higher Levels: When you cast this spell using a spell
     slot of 4th level or higher, the damage increases by 1d6 for each slot level 
    above 3rd.
    """
    name = "Fireball"
    level = 3
    casting_time = "1 action"
    casting_range = "150 feet"
    components = ('V', 's', 'm')
    materials = """A tiny ball of bat guano and sulfur"""
    duration = "Instantaneous"
    magic_school = "Evocation"
    classes = ('Sorcerer', 'Wizard')




In [44]:
print(b.as_dungeonsheets_class())

class Fireball(Spell):
    """A bright streak flashes from your pointing finger to a point you choose within 
    range then blossoms with a low roar into an explosion of flame. Each creature in
     a 20-foot radius sphere centered on that point must make a Dexterity saving 
    throw. A target takes 8d6 fire damage on a failed save, or half as much damage 
    on a successful one.
    
    The fire spreads around corners. It ignites flammable 
    objects in the area that aren't being worn or carried.
    
    At Higher Levels: When 
    you cast this spell using a spell slot of 4th level or higher, the damage 
    increases by 1d6 for each slot level above 3rd.
    """
    name = "Fireball"
    level = 3
    casting_time = "1 action"
    casting_range = "150 feet"
    components = ('V', 'S', 'M')
    materials = """A tiny ball of bat guano and sulfur"""
    duration = "Instantaneous"
    magic_school = "Evocation"
    classes = ('Sorcerer', 'Wizard')


